<a href="https://colab.research.google.com/github/AmarnathMishra777/Network-File-Sharing-Server-Client/blob/main/Network_File_Sharing_Server_%26_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile server.py
import socket, threading, os, ssl, json, hashlib, secrets

HOST = "127.0.0.1"
PORT = 9000
USER_FILE = "users.json"
SHARED_DIR = "shared_files"
CERT_FILE = "server_cert.pem"
KEY_FILE = "server_key.pem"
ITERATIONS = 150000

os.makedirs(SHARED_DIR, exist_ok=True)

def load_users():
    if not os.path.exists(USER_FILE): return {}
    with open(USER_FILE, "r") as f: return json.load(f)

def verify_password(stored, password):
    salt = bytes.fromhex(stored["salt"])
    correct_hash = bytes.fromhex(stored["hash"])
    test_hash = hashlib.pbkdf2_hmac("sha256", password.encode(), salt, ITERATIONS)
    return secrets.compare_digest(correct_hash, test_hash)

def recv_line(conn):
    data = b""
    while True:
        ch = conn.recv(1)
        if not ch or ch == b"\n": break
        data += ch
    return data.decode().strip()

def send_line(conn, text):
    conn.sendall((text + "\n").encode())

def handle_client(conn, addr, users):
    print(f"[+] Connected: {addr}")
    authenticated = False
    while True:
        data = recv_line(conn)
        if not data: break
        parts = data.split(" ", 2)
        cmd = parts[0].upper()
        if cmd == "AUTH":
            if len(parts) < 3: send_line(conn, "ERR Missing creds"); continue
            u, p = parts[1], parts[2]
            if u in users and verify_password(users[u], p):
                authenticated = True; send_line(conn, "OK Authenticated")
            else: send_line(conn, "ERR Invalid credentials")
        elif cmd == "LIST":
            if not authenticated: send_line(conn, "ERR Not authenticated"); continue
            for f in os.listdir(SHARED_DIR): send_line(conn, f)
            send_line(conn, ".")
        elif cmd == "PUT":
            if not authenticated: send_line(conn, "ERR Not authenticated"); continue
            name, size = parts[1], int(parts[2]); send_line(conn, "OK")
            with open(os.path.join(SHARED_DIR, name), "wb") as f:
                rem = size
                while rem>0:
                    chunk = conn.recv(min(4096, rem))
                    if not chunk: break
                    f.write(chunk); rem-=len(chunk)
            send_line(conn, "OK")
        elif cmd == "GET":
            if not authenticated: send_line(conn, "ERR Not authenticated"); continue
            name = parts[1]
            path = os.path.join(SHARED_DIR, name)
            if not os.path.exists(path): send_line(conn, "ERR No file"); continue
            size = os.path.getsize(path); send_line(conn, f"SIZE {size}")
            ack = recv_line(conn)
            if ack!="OK": continue
            with open(path,"rb") as f:
                while (chunk:=f.read(4096)): conn.sendall(chunk)
        elif cmd == "QUIT":
            send_line(conn,"BYE"); break
        else: send_line(conn,"ERR Unknown cmd")
    conn.close()

def main():
    users = load_users()
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
    context.load_cert_chain(CERT_FILE, KEY_FILE)
    s = socket.socket(); s.bind((HOST,PORT)); s.listen(5)
    print(f"[i] Server on {HOST}:{PORT}")
    while True:
        raw, addr = s.accept()
        conn = context.wrap_socket(raw, server_side=True)
        threading.Thread(target=handle_client,args=(conn,addr,users),daemon=True).start()

if __name__=="__main__": main()


Overwriting server.py


In [4]:
%%writefile client.py
import socket, ssl, os

def recv_line(conn):
    data=b""
    while True:
        ch=conn.recv(1)
        if not ch or ch==b"\n": break
        data+=ch
    return data.decode().strip()

def send_line(conn,text):
    conn.sendall((text+"\n").encode())

def main():
    host="127.0.0.1"; port=9000
    ctx=ssl.create_default_context(); ctx.check_hostname=False; ctx.verify_mode=ssl.CERT_NONE
    with socket.create_connection((host,port)) as sock:
        with ctx.wrap_socket(sock,server_hostname=host) as conn:
            print("[+] Connected to server")
            send_line(conn,"AUTH user pass")
            print(recv_line(conn))
            send_line(conn,"LIST")
            while True:
                line=recv_line(conn)
                if line==".": break
                print("File:",line)
            send_line(conn,"QUIT")
            print(recv_line(conn))

if __name__=="__main__": main()


Writing client.py


In [5]:
# Generate certificate and key
!openssl req -x509 -newkey rsa:2048 -nodes -keyout server_key.pem -out server_cert.pem -days 1 -subj "/CN=localhost"

# Create user credentials
import json, os, hashlib, secrets
USER_FILE="users.json"; ITER=150000
def load_users(): return json.load(open(USER_FILE)) if os.path.exists(USER_FILE) else {}
def save_users(u): json.dump(u,open(USER_FILE,"w"),indent=2)
salt=secrets.token_bytes(16)
pwd="pass"
hashed=hashlib.pbkdf2_hmac("sha256",pwd.encode(),salt,ITER)
users=load_users(); users["user"]={"salt":salt.hex(),"hash":hashed.hex()}
save_users(users)
print("✅ User 'user' with password 'pass' created.")


..+.....+...+...............+.......+..+...+....+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*............+.+..+.......+.....+...+...+.+...........+............+.+...............+..+...+......+......+.......+..+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
...+...............+...+....+.....+.......+.........+.....+.+.........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*......+....+...+........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+.+......+..+...+.........+.............+..+.............+..+....+.....+.........+.+.....+......+.+.........+...............+...+..+...+...+....+.....+.............+...+.....+............+.............+...............+...+............+.........+...+..+......+.............+.....+....+..+...+.......+...+.....+.+..+...+.........+.+..+.......+...+...+......+.....+...+.............+..+................+...

In [6]:
import subprocess, time
server = subprocess.Popen(["python", "server.py"])
time.sleep(2)  # give server time to start
print("✅ Server started!")


✅ Server started!


In [7]:
!python client.py


[+] Connected to server
OK Authenticated
BYE


In [8]:
server.terminate()
print("🛑 Server stopped.")


🛑 Server stopped.
